In [1]:
!pip install -q langchain langchain-community google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00


In [2]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [8]:
import os
from google.colab import userdata

# Access the API key from Colab's Secrets Manager
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Simple chain

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

In [15]:
from IPython.display import display

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


# build the prompt : step1

prompt = PromptTemplate(
    template="generate 2 intersting facts about {topic}",
    input_variables=["topic"]
)

# build the model: step2


# use parser for result generation: step3
parser = StrOutputParser()


# connect all of this steps using the | to form chain...
chain = prompt | model | parser

result = chain.invoke({"topic":"football"})

print(result)


Here are 2 interesting facts about football (soccer):

1.  **The Offside Rule Was Once Much Stricter:** In the early days of football, a player was considered offside if they were ahead of the ball at all, similar to rugby. The modern offside rule, which requires at least two opponents (including the goalkeeper) to be between an attacking player and the goal line when the ball is played, was gradually introduced. A significant change in 1925 reduced the number of opponents needed from three to two, which drastically increased goal-scoring and fundamentally changed the attacking dynamics of the game.

2.  **A Goalkeeper Holds the Record for Most Goals Scored by a Goalkeeper:** While goalkeepers are primarily known for preventing goals, Brazilian legend Rogério Ceni holds the astonishing record for the most goals scored by a goalkeeper in professional football. During his career with São Paulo FC, he netted an incredible 131 goals, primarily from free-kicks and penalties, making him a un

In [17]:
!pip install grandalf
chain.get_graph().print_ascii()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.6 MB/s eta 0:00:00
      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


# Sequential chain

In [18]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt1 = PromptTemplate(
    template="generate a simple short report on {topic}",
    input_variables=["topic"]
)

prompt2 = PromptTemplate(
    template="generate 2 points summary for the following text \n {text}",
    input_variables=["text"]
)

# model

parser = StrOutputParser()


chain = prompt1 | model | parser | prompt2 | model | parser

result = chain.invoke({"topic":"umemployment in Nepal"})

display(result)

'Here are two points summarizing the text:\n\n1.  Unemployment and underemployment are major socio-economic challenges in Nepal, leading to a "brain drain" of skilled youth seeking jobs abroad, economic hardship, and heavy reliance on remittances.\n2.  The problem is driven by limited job creation, a mismatch between skills and market demands, and slow industrial growth, necessitating a multi-faceted approach including vocational training, entrepreneurship promotion, and education reform.'

In [19]:
chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *      

# Parallel chain :
user provides a document and generate notes and quiz and combines them and show to user.
docs----first model -----> notes --------|
 |same - docs______sec model ------->quiz ----------|(combined both o/p)
                                         |----model 3-----> output

In [30]:
%pip install -q mistralai
%pip install -q langchain-mistralai

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel # to execute multiple parallel chain

from langchain_google_genai import ChatGoogleGenerativeAI
model1 = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

from langchain_mistralai import ChatMistralAI
mistral_model = ChatMistralAI(model="mistral-small")



prompt1 = PromptTemplate(
    template="generate a simple very short notes on {text}",
    input_variables=["text"]

)

prompt2 = PromptTemplate(
    template="genarate a short question answer for the following text \n {text}",
    input_variables=["text"]
)

prompt3 = PromptTemplate(
    template="merge the provided notes and quiz into single document \n notes-> {notes} \n quiz -> {quiz}",
    input_variables=["notes","quiz"]
)

parser = StrOutputParser()

parallel_chain = RunnableParallel(
    {

     "notes": prompt1 | model1 | parser,
     "quiz":prompt2 | mistral_model | parser
    }
)


merge_chain = prompt3 | mistral_model | parser

chain = parallel_chain | merge_chain

text = """ONNX, which stands for Open Neural Network Exchange, is an open standard format for representing machine learning models.
It allows models trained in one framework (like PyTorch or TensorFlow) to be easily exported and used in other frameworks or environments without significant modifications."""
result = chain.invoke({"text":text})

display(result)

"ONNX (Open Neural Network Exchange) Notes and Quiz:\n\nONNX is an open standard format for representing machine learning models. It enables models trained in one framework, such as PyTorch or TensorFlow, to be easily exported and used in other environments. This cross-framework compatibility is crucial in ensuring that machine learning models can be shared and utilized across different platforms and tools.\n\nQuiz:\n\nQ: What does ONNX stand for?\nA: ONNX stands for Open Neural Network Exchange. It's an open standard format for representing machine learning models, allowing models trained in one framework to be exported and used in other frameworks without significant modifications."

In [31]:
chain.get_graph().print_ascii()

                  +---------------------------+                
                  | Parallel<notes,quiz>Input |                
                  +---------------------------+                
                     ****               ****                   
                  ***                       ***                
                **                             **              
    +----------------+                    +----------------+   
    | PromptTemplate |                    | PromptTemplate |   
    +----------------+                    +----------------+   
             *                                      *          
             *                                      *          
             *                                      *          
+------------------------+                 +---------------+   
| ChatGoogleGenerativeAI |                 | ChatMistralAI |   
+------------------------+                 +---------------+   
             *                          

# setup for the mistral AI is done here 😊

In [23]:
%pip install -q mistralai

Now, let's set up your Mistral API key. We'll use Colab's Secrets Manager for secure storage.

In [24]:
from google.colab import userdata
import os

os.environ["MISTRAL_API_KEY"] = userdata.get("MISTRAL_API_KEY")

Finally, let's initialize the Mistral model.

In [28]:
from langchain_mistralai import ChatMistralAI

mistral_model = ChatMistralAI(model="mistral-small") # You can change the model name if needed

In [27]:
%pip install -q langchain-mistralai

In [29]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


# build the prompt : step1

prompt = PromptTemplate(
    template="generate 2 intersting facts about {topic}",
    input_variables=["topic"]
)

# build the model: step2


# use parser for result generation: step3
parser = StrOutputParser()


# connect all of this steps using the | to form chain...
chain = prompt | mistral_model | parser

result = chain.invoke({"topic":"football"})

print(result)


1. Football, or soccer as it's known in some countries, is the most popular sport in the world. According to FIFA, the sport's international governing body, there are more than 240 million players actively involved in football around the globe.

2. The longest football match in history took place in 1986 in Buenos Aires, Argentina between two amateur teams, San Lorenzo de Almagro and Huracán de Montemar. The match ended in a 2-1 victory for San Lorenzo after a whopping 23 hours, 14 minutes of play. The match was played over four days with several intermissions for players to rest and refuel.


# Conditional chains : give feedback according to the feedback after analyzing the sentiment

In [37]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel , RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

from typing import Literal

from langchain_mistralai import ChatMistralAI
mistral_model = ChatMistralAI(model="mistral-small")

parser = StrOutputParser()


class Feedback(BaseModel):

  sentiment: Literal['positive', 'negative'] = Field(description="give the sentiment of the parser")


parser2 = PydanticOutputParser(pydantic_object=Feedback)



prompt1 = PromptTemplate(
    template="classify the sentiment of the following feedback text into 'positive' or 'negative' only: \n {feedback} \n {format_instructions}",
    input_variables=['feedback'],
    partial_variables={"format_instructions":parser2.get_format_instructions()}
)


classifier_chain = prompt1 | mistral_model | parser2

# result = classifier_chain.invoke({"feedback":"The food was delicious"}).sentiment
# print(result)
# sentiment = 'positive' <-- output

# write prompt for each branch


prompt2 = PromptTemplate(
    template="write an appropriate response to this positive feedback \n {feedback}",
    input_variables=["feedback"]
)

prompt3 = PromptTemplate(
    template="write an appropriate response to this negative feedback \n {feedback}",
    input_variables=["feedback"]
)

branch_chain = RunnableBranch(
     (lambda x: x.sentiment == 'positive', prompt2 | mistral_model | parser),
    (lambda x: x.sentiment == 'negative', prompt3 | mistral_model |parser),
    RunnableLambda( lambda x:"could not find sentiment")
)


chain = classifier_chain | branch_chain

result = chain.invoke({"feedback":" this is a terrible phone"})
display(result)

'Dear Customer,\n\nWe are sorry to hear that your experience with us was not a positive one. Our goal is to provide excellent service and products to all of our customers, and it seems that we fell short during your interaction with us.\n\nWe appreciate your feedback and would like to take the opportunity to make things right. Please contact us directly so that we can discuss your concerns and work towards a resolution. Our customer service team is available to assist you and can be reached at [insert contact information].\n\nOnce again, we apologize for any inconvenience and thank you for bringing this matter to our attention. We value your business and hope to have the opportunity to serve you better in the future.\n\nBest regards,\n[Your Name]\n[Your Position]\n[Company Name]'

In [38]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
    +---------------+    
    | ChatMistralAI |    
    +---------------+    
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
